In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("..")

In [23]:
import torch
from torch.autograd import Variable
from utilities_functions.intermediate_layer_extraction import return_layer_input
import deepmatcher as dm 
from utilities_functions.ri_calculator import find_smallest_variation_to_change

In [24]:
hybrid_model = dm.MatchingModel(attr_summarizer = "hybrid")
hybrid_model.load_state('../models/amazongoogle_hybrid.pth')
hybrid_model = hybrid_model.cuda()

In [ ]:
datasets = dm.data.process('../Structured/itunes-amazon/',train='merged_train.csv',validation='merged_valid.csv',
                                       test='merged_test.csv',left_prefix='ltable_',right_prefix='rtable_')

In [7]:
layer_inputs, neg_ids = return_layer_input("../Structured/Amazon-Google/","test_negatives",128,
                                           hybrid_model,hybrid_model.classifier,device=-1)

In [15]:
layer_inputs_onCuda = []
for layer_inp in layer_inputs:
    layer_inp = layer_inp.cuda()
    layer_inputs_onCuda.append(layer_inp)

In [21]:
attributes = ['title','manufacturer','price']

In [26]:
current_sample = 0
#each column of this matrix is related to a specific attribute
negatives_ri_matrix = []
for batch in layer_inputs_onCuda:
    for sample_index in range(len(batch)):
        print('Processing sample number {}'.format(current_sample))
        current_sample_ris = list(map(lambda att: find_smallest_variation_to_change(hybrid_model.classifier,
                                                                                    classifier_length=900,
                                                                                    attribute_length=300,
                                                                                    input_matrix=batch,
                                                                                    vector_index=sample_index,
                                                                                    attributes=[attributes.index(att)]
                                                                                    ,class_to_reach=1),attributes))
        negatives_ri_matrix.append(current_sample_ris)
        current_sample+=1

Processing sample number 0
Processing sample number 1
Processing sample number 2
Processing sample number 3
Processing sample number 4
Processing sample number 5
Processing sample number 6
Processing sample number 7
Processing sample number 8
Processing sample number 9
Processing sample number 10
Processing sample number 11
Processing sample number 12
Processing sample number 13
Processing sample number 14
Processing sample number 15
Processing sample number 16
Processing sample number 17
Processing sample number 18
Processing sample number 19
Processing sample number 20
Processing sample number 21
Processing sample number 22
Processing sample number 23
Processing sample number 24
Processing sample number 25
Processing sample number 26
Processing sample number 27
Processing sample number 28
Processing sample number 29
Processing sample number 30
Processing sample number 31
Processing sample number 32
Processing sample number 33
Processing sample number 34
Processing sample number 35
Pr

KeyboardInterrupt: 

In [ ]:
prediction = hybrid_model.classifier.forward(batches[0])

In [ ]:
batches[0].register_hook(print)

In [ ]:
prediction[0].backward(torch.cuda.FloatTensor([0,1]))